# YOLO 토마토 질병 분류 모델학습

## 환경설정

In [ ]:
# 1단계: 환경 설정
print("🚀 YOLOv8 토마토 질병 분류 - 학습 노트북")
print("=" * 50)

# 필요한 패키지 설치
print("📦 필요한 패키지 설치 중...")
!pip install ultralytics -q
print("✅ ultralytics 설치 완료")

# 라이브러리 import
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import drive

# 구글 드라이브 마운트
print("\n📁 구글 드라이브 마운트 중...")
drive.mount('/content/drive', force_remount=True)
print("✅ 구글 드라이브 마운트 완료")

# GPU 및 시스템 정보 확인
print(f"\n💻 시스템 정보:")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


## 데이터셋 및 YAML 파일 확인

In [ ]:
# 2단계: 데이터셋 및 YAML 파일 확인
print(f"\n=== 데이터셋 확인 ===")

dataset_path = "/content/drive/MyDrive/dataset_6class_complete"
yaml_path = "/content/drive/MyDrive/data_6class.yaml"

print(f"데이터셋 경로: {dataset_path}")
print(f"YAML 경로: {yaml_path}")

# 데이터셋 구조 확인
if os.path.exists(dataset_path):
    print(f"✅ 데이터셋 존재 확인")
    for part in ['newpart_01', 'newpart_02', 'newpart_03']:
        part_dir = f"{dataset_path}/{part}"
        if os.path.exists(part_dir):
            img_count = len([f for f in os.listdir(part_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
            txt_count = len([f for f in os.listdir(part_dir) if f.endswith('.txt')])
            role = "Train" if part != 'newpart_03' else "Val"
            print(f"  {part}: 이미지 {img_count}개, 라벨 {txt_count}개 ({role})")
else:
    print(f"❌ 데이터셋이 없습니다: {dataset_path}")
    print("⚠️ 학습을 진행할 수 없습니다. 데이터셋을 확인해주세요.")

# YAML 파일 확인
if os.path.exists(yaml_path):
    print(f"✅ YAML 파일 존재 확인")
    with open(yaml_path, 'r', encoding='utf-8') as f:
        yaml_content = f.read()
        print(f"YAML 내용:\n{yaml_content}")
else:
    print(f"❌ YAML 파일이 없습니다: {yaml_path}")
    print("⚠️ 학습을 진행할 수 없습니다. YAML 파일을 확인해주세요.")


## 학습 실행 여부 확인

In [ ]:
# 3단계: 학습 실행 여부 확인 및 안전장치
def check_existing_training():
    """기존 학습 결과 확인 - 중복 학습 방지"""
    results_dir = '/content/drive/MyDrive/yolo_training_results/tomato_6class_v1'
    weights_dir = f"{results_dir}/weights"

    if os.path.exists(weights_dir):
        weights = [f for f in os.listdir(weights_dir) if f.endswith('.pt')]
        if weights:
            print(f"⚠️ 기존 학습 결과가 발견되었습니다!")
            print(f"📂 위치: {weights_dir}")
            print(f"🔍 모델 파일들: {weights}")

            print(f"\n🤔 다음 중 선택하세요:")
            print(f"1. 새로 학습하기 (기존 결과 덮어쓰기)")
            print(f"2. 기존 모델 사용하기 (학습 건너뛰기)")
            print(f"3. 학습 결과만 분석하기")

            return True
    return False

# 기존 학습 확인
existing_training = check_existing_training()

## YOLOv8 학습 시작

In [ ]:
# 4단계: YOLOv8 학습 시작
def start_training(force_retrain=False):
    """YOLO 모델 학습 실행"""

    if existing_training and not force_retrain:
        print(f"\n🛑 기존 학습 결과가 있습니다.")
        print(f"💡 새로 학습하려면 start_training(force_retrain=True)를 실행하세요.")
        return None

    print(f"\n=== YOLOv8 학습 시작 ===")

    if not os.path.exists(dataset_path) or not os.path.exists(yaml_path):
        print(f"❌ 데이터셋 또는 YAML 파일이 없습니다. 학습을 중단합니다.")
        return None

    # GPU 메모리 정리
    torch.cuda.empty_cache()

    # YOLOv8s 모델 로드
    model = YOLO('yolov8s.pt')
    print("✅ YOLOv8s 사전 훈련 모델 로드 완료")

    # 학습 시작
    print(f"\n🚀 학습 시작! (예상 소요 시간: 30-60분)")
    print(f"📊 학습 진행 상황은 실시간으로 표시됩니다.")

    results = model.train(
        # 데이터 설정
        data=yaml_path,

        # 학습 파라미터
        epochs=50,              # 50 에포크
        imgsz=640,              # 이미지 크기
        batch=16,               # 배치 크기
        device=0,               # GPU 사용

        # 출력 설정 (구글 드라이브에 저장)
        project='/content/drive/MyDrive/yolo_training_results',
        name='tomato_6class_v1',

        # 최적화 파라미터
        optimizer='AdamW',       # 옵티마이저
        lr0=0.01,               # 초기 학습률
        warmup_epochs=3,        # 웜업 에포크

        # validation 관련
        conf=0.001,             # validation confidence threshold
        iou=0.6,                # IoU threshold

        # 손실 함수 가중치
        box=7.5,                # 박스 손실 가중치
        cls=0.5,                # 분류 손실 가중치
        dfl=1.5,                # DFL 손실 가중치

        # 저장 및 로그 설정
        save=True,              # 모델 저장
        save_period=10,         # 10 에포크마다 저장
        plots=True,             # 그래프 저장
        val=True,               # 검증 수행

        # 조기 종료 및 기타
        patience=20,            # 조기 종료 기준
        exist_ok=True,          # 기존 폴더 덮어쓰기 허용
        verbose=True            # 상세 로그
    )

    print(f"\n🎉 학습 완료!")
    return results

## 학습 결과 확인 함수

In [ ]:
# 5단계: 학습 결과 확인 함수
def check_training_results():
    """학습 결과 기본 확인"""
    print(f"\n=== 학습 결과 확인 ===")

    results_dir = '/content/drive/MyDrive/yolo_training_results/tomato_6class_v1'
    weights_dir = f"{results_dir}/weights"

    if os.path.exists(weights_dir):
        weights = [f for f in os.listdir(weights_dir) if f.endswith('.pt')]
        print(f"✅ 저장된 가중치: {weights}")

        # best.pt 모델로 최종 검증
        if 'best.pt' in weights:
            best_model_path = f"{weights_dir}/best.pt"
            print(f"🏆 최고 성능 모델: {best_model_path}")

            # 최종 모델 로드 및 검증
            best_model = YOLO(best_model_path)

            # Validation 실행
            val_results = best_model.val(
                data=yaml_path,
                conf=0.001,
                iou=0.6,
                device=0
            )

            print(f"\n📊 최종 성능:")
            print(f"mAP@0.5: {val_results.box.map50:.4f}")
            print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")

            # 클래스별 성능
            class_names = ['정상', '토마토잿빛곰팡이병', '토마토흰가루병',
                          '다량원소결핍(N)', '다량원소결핍(P)', '다량원소결핍(K)']

            if hasattr(val_results.box, 'ap') and len(val_results.box.ap) > 0:
                print(f"\n📈 클래스별 mAP@0.5:")
                for i, name in enumerate(class_names):
                    if i < len(val_results.box.ap):
                        ap_value = val_results.box.ap[i] if val_results.box.ap[i] is not None else 0.0
                        print(f"  {name}: {ap_value:.4f}")

        print(f"\n📂 결과 저장 위치: {results_dir}")
        print(f"   - 가중치: {weights_dir}")
        print(f"   - 그래프 및 로그: {results_dir}")

    else:
        print(f"❌ 학습 결과를 찾을 수 없습니다.")
        print(f"💡 학습을 먼저 실행해주세요: start_training()")


## 학습 결과 상세 분석

In [ ]:
# 6단계: 학습 결과 상세 분석
def analyze_training_results():
    """학습 결과 상세 분석 - 과적합 여부 판단"""
    print("📊 학습 결과 상세 분석 시작!")

    results_dir = '/content/drive/MyDrive/yolo_training_results/tomato_6class_v1'
    results_csv = f"{results_dir}/results.csv"

    if not os.path.exists(results_csv):
        print(f"❌ results.csv 파일이 없습니다: {results_csv}")
        print(f"💡 학습을 먼저 완료해주세요.")
        return None

    # CSV 읽기
    df = pd.read_csv(results_csv)
    print(f"✅ 총 {len(df)}개 에포크 학습 완료")

    # mAP 트렌드 분석
    map50_col = None
    if 'metrics/mAP50(B)' in df.columns:
        map50_col = 'metrics/mAP50(B)'
    elif 'val/mAP50' in df.columns:
        map50_col = 'val/mAP50'

    if map50_col:
        map50_values = df[map50_col].values
        best_map50 = max(map50_values)
        best_epoch = df[df[map50_col] == best_map50].index[0] + 1
        last_map50 = map50_values[-1]

        print(f"\n📊 성능 요약:")
        print(f"  최고 mAP50: {best_map50:.4f} (에포크 {best_epoch})")
        print(f"  최종 mAP50: {last_map50:.4f}")
        print(f"  성능 차이: {(best_map50 - last_map50):.4f}")

        # 과적합 판단
        if best_map50 - last_map50 > 0.02:
            print("🚨 과적합 의심: 최고 성능 대비 2% 이상 하락")
            print("💡 권장: best.pt 모델 사용")
        elif best_map50 - last_map50 > 0.01:
            print("⚠️ 경미한 과적합 가능성: 1-2% 하락")
        else:
            print("✅ 과적합 없음: 성능 안정적")

    return df

## 학습 곡선 시각화

In [ ]:
# 7단계: 학습 곡선 시각화
def plot_training_curves(df=None):
    """학습 곡선 시각화"""
    if df is None:
        results_csv = '/content/drive/MyDrive/yolo_training_results/tomato_6class_v1/results.csv'
        if not os.path.exists(results_csv):
            print(f"❌ 학습 결과가 없습니다.")
            return
        df = pd.read_csv(results_csv)

    print(f"\n=== 학습 곡선 시각화 ===")

    # 전문적인 4개 서브플롯 구성
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Training Results Summary', fontsize=16, fontweight='bold')

    epochs = range(len(df))

    # 1. mAP50 진행
    if 'metrics/mAP50(B)' in df.columns:
        ax1 = axes[0, 0]
        ax1.plot(epochs, df['metrics/mAP50(B)'], color='#1f77b4', linewidth=2, label='mAP50')
        ax1.set_title('mAP50 진행', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('mAP50')
        ax1.grid(True, alpha=0.3)
        ax1.legend()

        # 최고점 표시
        best_idx = df['metrics/mAP50(B)'].idxmax()
        best_value = df['metrics/mAP50(B)'].iloc[best_idx]
        ax1.plot(best_idx, best_value, 'ro', markersize=8)
        ax1.text(best_idx, best_value + 0.02, f'Best: {best_value:.3f}',
                ha='center', fontweight='bold')

    # 2. mAP50-95 진행
    if 'metrics/mAP50-95(B)' in df.columns:
        ax2 = axes[0, 1]
        ax2.plot(epochs, df['metrics/mAP50-95(B)'], color='#2ca02c', linewidth=2, label='mAP50-95')
        ax2.set_title('mAP50-95 진행', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('mAP50-95')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

    # 3. Loss 진행
    ax3 = axes[1, 0]
    train_loss_cols = [col for col in df.columns if 'train' in col and 'loss' in col]
    val_loss_cols = [col for col in df.columns if 'val' in col and 'loss' in col]

    if train_loss_cols and val_loss_cols:
        ax3.plot(epochs, df[train_loss_cols[0]], color='#d62728', linewidth=2, label='Train Loss')
        ax3.plot(epochs, df[val_loss_cols[0]], color='#ff7f0e', linewidth=2, label='Val Loss')
        ax3.set_title('Loss 진행', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Loss')
        ax3.grid(True, alpha=0.3)
        ax3.legend()

    # 4. 최근 성능 상세
    ax4 = axes[1, 1]
    if 'metrics/mAP50(B)' in df.columns:
        recent_epochs = min(10, len(df))
        recent_data = df['metrics/mAP50(B)'].tail(recent_epochs)
        recent_x = range(len(df) - recent_epochs, len(df))

        ax4.plot(recent_x, recent_data, 'o-', color='#9467bd', linewidth=2,
                markersize=6, label=f'Recent {recent_epochs} epochs')
        ax4.set_title(f'최근 {recent_epochs} 에포크 상세', fontsize=14, fontweight='bold')
        ax4.set_xlabel('Epoch')
        ax4.set_ylabel('mAP50')
        ax4.grid(True, alpha=0.3)
        ax4.legend()

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/training_analysis_complete.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"📈 학습 곡선 저장: /content/drive/MyDrive/training_analysis_complete.png")


## 간단한 추론 테스트

In [ ]:
# 8단계: 간단한 추론 테스트
def quick_inference_test():
    """학습 완료 후 간단한 추론 테스트"""
    print(f"\n=== 간단한 추론 테스트 ===")

    best_model_path = '/content/drive/MyDrive/yolo_training_results/tomato_6class_v1/weights/best.pt'

    if not os.path.exists(best_model_path):
        print(f"❌ 모델이 없습니다. 학습을 먼저 완료해주세요.")
        return

    model = YOLO(best_model_path)

    # 테스트 이미지 선택
    val_dir = "/content/drive/MyDrive/dataset_6class_complete/newpart_03"
    if os.path.exists(val_dir):
        val_images = [f for f in os.listdir(val_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        if val_images:
            test_image = f"{val_dir}/{val_images[0]}"
            print(f"📷 테스트 이미지: {val_images[0]}")

            # 추론 실행
            results = model(test_image, conf=0.25, device=0)

            # 결과 출력
            for r in results:
                if r.boxes is not None and len(r.boxes) > 0:
                    print(f"✅ {len(r.boxes)}개 객체 검출 성공!")
                    for i, box in enumerate(r.boxes):
                        class_id = int(box.cls[0])
                        confidence = float(box.conf[0])
                        class_name = model.names[class_id]
                        print(f"  {i+1}. {class_name} (신뢰도: {confidence:.3f})")
                else:
                    print(f"❌ 검출된 객체 없음")

            print(f"🎯 상세한 추론 테스트는 Inference 노트북을 사용하세요!")